In [116]:
#Triple Encryption using AES,Blowfish and DES. The key of AES is encrypted using RSA.

In [117]:
import time
start_time = time.time()
import numpy as np
import blowfish
import rsa
from Cryptodome.Cipher import AES
from Cryptodome.Cipher import DES
from PIL import Image
import cv2
import math

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [118]:
image = cv2.imread('bird.jpg', 1)
cv2.imshow('original', image)

In [119]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()

In [120]:
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)

In [121]:
key = b'aespasswordisnow'
cipher_a = AES.new(key, AES.MODE_EAX)
nonce = cipher_a.nonce
ciphertext_enc, tag = cipher_a.encrypt_and_digest(img_str)

In [122]:
public_key, private_key = rsa.newkeys(1024)

In [123]:
sig = rsa.sign(key, private_key, 'SHA-256')
enc_key = rsa.encrypt(key, public_key)

In [124]:
img_str_np_enc_aes = np.frombuffer(ciphertext_enc, dtype=np.uint8)
a = len(img_str_np_enc_aes)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2


img_str_np_enc_aes1 = img_str_np_enc_aes.reshape(i,len(img_str_np_enc_aes)//i)

In [125]:
enc_img_aes = Image.fromarray(img_str_np_enc_aes1, 'L')
enc_img_aes.show(title='aes encrypted')

In [126]:
cipher_b = blowfish.Cipher(b'blowfishpasswordishere')

In [127]:
data = img_str # electronic codebook mode

data_encrypted = b"".join(cipher_b.encrypt_ecb(ciphertext_enc))

In [128]:
img_str_np_enc_blow = np.frombuffer(data_encrypted, dtype=np.uint8)
a = len(img_str_np_enc_blow)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2


img_str_np_enc_blow1 = img_str_np_enc_blow.reshape(i,len(img_str_np_enc_blow)//i)

In [129]:
enc_img_blow = Image.fromarray(img_str_np_enc_blow1, 'L')
enc_img_blow.show(title='blowfish encrypted')

In [130]:
key_d = b'despasci'
cipher_d = DES.new(key_d, DES.MODE_EAX)
nonce_d = cipher_d.nonce
msg_des = cipher_d.encrypt(data_encrypted)

In [131]:
cipher_d = DES.new(key_d, DES.MODE_EAX, nonce_d)
msg_des_dec = cipher_d.decrypt(msg_des)

assert msg_des_dec == data_encrypted

In [132]:
img_str_np_enc_des = np.frombuffer(msg_des_dec, dtype=np.uint8)
a = len(img_str_np_enc_des)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2

img_str_np_enc_des1 = img_str_np_enc_des.reshape(i,len(img_str_np_enc_des)//i)

In [133]:
enc_img_des = Image.fromarray(img_str_np_enc_des1, 'L')
enc_img_des.show(title='des encrypted')

In [134]:
data_decrypted = b"".join(cipher_b.decrypt_ecb(data_encrypted))

#assert data == data_decrypted

In [135]:
dec_key = rsa.decrypt(enc_key, private_key)
rsa.verify(dec_key, sig, public_key)

'SHA-256'

In [136]:
cipher_a = AES.new(dec_key, AES.MODE_EAX, nonce=nonce)
plaintext_enc = cipher_a.decrypt(data_decrypted)
try:
    cipher_a.decrypt(ciphertext_enc)
    print("AES decryption successful")
except ValueError:
    print("Decryption failed")

AES decryption successful


In [137]:
img_str_np = np.frombuffer(plaintext_enc, dtype=np.uint8)
img_str_np

array([255, 216, 255, ...,   0,   0,   0], dtype=uint8)

In [138]:
img = cv2.imdecode(img_str_np, 1)

In [139]:
cv2.imshow('decrypted', img)

In [140]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

d = psnr(image, img)
print(d)

43.22586472210333


In [141]:
cv2.PSNR(image, img)

42.722213649793986

In [142]:
Y = np.square(np.subtract(image,img)).mean()
print("MSE:", Y)

MSE: 3.0938117793787896


In [143]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99943421],
       [0.99943421, 1.        ]])

In [144]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 10.593448877334595 seconds ---
